In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [105]:
import pandas
import pygrib
import pickle
import os.path

In [114]:

def extract_grib_files(grib_dir, out_dir, year, min_lat, max_lat, max_lng, min_lng):
    
    max_lat_ndx = None
    min_lat_ndx = None
    max_lng_ndx = None
    min_lng_ndx = None
    for day in range(1,25):
        for fctime in [0,6,12,18]:
            for simulation_time in range(0,6):
                grib_file = f"{grib_dir}/{year}/{year}07{day:02}-{fctime:02}-gfs.t{fctime:02}z.pgrb2.0p25.f{simulation_time:03}"
                out_file = f"{out_dir}/{year}/{year}07{day:02}-{fctime:02}-gfs.0p25.f{simulation_time:03}.pkl"
                print(f"{grib_file} => {out_file}")
                if not os.path.exists(grib_file):
                    print("grib file not found, exiting")
                    return
                    
                if os.path.exists(out_file):
                    print("  file exists")
                else:
                    print("  extracting")
                    with pygrib.open(grib_file) as grbs:
                        u_msg = grbs.select(name="10 metre U wind component", level=10, forecastTime=simulation_time)[0].values
                        v_msg = grbs.select(name="10 metre V wind component", level=10, forecastTime=simulation_time)[0].values
                        lats, lngs = grbs[1].latlons()
                    
                        
                        sub_array = {}
                        if max_lat_ndx is None:
                            print("calcuating bounds")
                            print(f"min_lat={min_lat}")
                            print(f"max_lat={max_lat}")
                            for lat_ndx in range(0, lats.shape[0]):
                                #print(lat_ndx, lats[lat_ndx][0])
                                if min_lat_ndx is None and lats[lat_ndx][0] <= max_lat:
                                    min_lat_ndx = lat_ndx-1
                                    print(f"min_lat_ndx={min_lat_ndx} lat={lats[min_lat_ndx][0]}")
                                if max_lat_ndx is None and lats[lat_ndx][0] <= min_lat:
                                    max_lat_ndx = lat_ndx
                                    print(f"max_lat_ndx={max_lat_ndx} lat={lats[max_lat_ndx][0]}")
                        
                                    break
                            min_lng = min_lng if min_lng>0 else min_lng+360
                            max_lng = max_lng if max_lng>0 else max_lng+360
                            print(f"min_lng={min_lng}")
                            print(f"max_lng={max_lng}")
                            
                            for lng_ndx in range(0, lngs.shape[1]):
                                #print(lng_ndx, lngs[0][lng_ndx])
                                if min_lng_ndx is None and lngs[0][lng_ndx] >= min_lng:
                                    min_lng_ndx = lng_ndx-1
                                    print(f"min_lng_ndx={min_lng_ndx} lng={lngs[0][min_lng_ndx]}")
                                if max_lng_ndx is None and lngs[0][lng_ndx] >= max_lng:
                                    max_lng_ndx = lng_ndx
                                    print(f"max_lng_ndx={max_lng_ndx} lng={lngs[0][max_lng_ndx]}")
                    
                        sub_array['lats']  = lats[min_lat_ndx:max_lat_ndx, min_lng_ndx:max_lng_ndx]
                        sub_array['lngs']  = lngs[min_lat_ndx:max_lat_ndx, min_lng_ndx:max_lng_ndx]
                        sub_array['u_msg'] = u_msg[min_lat_ndx:max_lat_ndx, min_lng_ndx:max_lng_ndx]
                        sub_array['v_msg'] = v_msg[min_lat_ndx:max_lat_ndx, min_lng_ndx:max_lng_ndx]
                        # Save the sub_array to a file
                        with open(out_file, 'wb') as file:
                            pickle.dump(sub_array, file)

In [115]:
min_lat = 21.1
max_lat = 34.1
max_lng = -118
min_lng = -158

In [ ]:
#grib_dir ='historical_gribs'
grib_dir = '/Volumes/Gribs'
out_dir = 'historical_wind_data'
year = 2024

extract_grib_files(grib_dir, out_dir, year, min_lat, max_lat, max_lng, min_lng)

/Volumes/Gribs/2024/20240701-00-gfs.t00z.pgrb2.0p25.f000 => historical_wind_data/2024/20240701-00-gfs.0p25.f000.pkl
  extracting
calcuating bounds
min_lat=21.1
max_lat=34.1
min_lat_ndx=223 lat=34.25
max_lat_ndx=276 lat=21.0
min_lng=202
max_lng=242
min_lng_ndx=807 lng=201.75
max_lng_ndx=968 lng=242.0
/Volumes/Gribs/2024/20240701-00-gfs.t00z.pgrb2.0p25.f001 => historical_wind_data/2024/20240701-00-gfs.0p25.f001.pkl
  extracting
/Volumes/Gribs/2024/20240701-00-gfs.t00z.pgrb2.0p25.f002 => historical_wind_data/2024/20240701-00-gfs.0p25.f002.pkl
  extracting
/Volumes/Gribs/2024/20240701-00-gfs.t00z.pgrb2.0p25.f003 => historical_wind_data/2024/20240701-00-gfs.0p25.f003.pkl
  extracting
/Volumes/Gribs/2024/20240701-00-gfs.t00z.pgrb2.0p25.f004 => historical_wind_data/2024/20240701-00-gfs.0p25.f004.pkl
  extracting
/Volumes/Gribs/2024/20240701-00-gfs.t00z.pgrb2.0p25.f005 => historical_wind_data/2024/20240701-00-gfs.0p25.f005.pkl
  extracting
/Volumes/Gribs/2024/20240701-06-gfs.t06z.pgrb2.0p25.f0